# Regression
Features = rates from selected diseases (FeatureSelection) + suicide rate 

Target = suicide rate of the following year

Import libraries

In [95]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from skopt import gp_minimize
import glob
root = "../../"

In [3]:
dict_uf_cod = {11: 'RO',
12: 'AC',
13: 'AM',
14: 'RR',
15: 'PA',
16: 'AP',
17: 'TO',
21: 'MA',
22: 'PI',
23: 'CE',
24: 'RN',
25: 'PB',
26: 'PE',
27: 'AL',
28: 'SE',
29: 'BA',
31: 'MG',
32: 'ES',
33: 'RJ',
35: 'SP',
41: 'PR',
42: 'SC',
43: 'RS',
50: 'MS',
51: 'MT',
52: 'GO',
53: 'DF'}

Prepare dataset

In [6]:
disease = ""
path = root + "CSV/TabNet/Internacoes_Rate/"
all_files = glob.glob(path + "*.csv")
suicide = pd.read_csv(root +'CSV/Suicide/suicide_rates_08_18.csv', index_col=0)

years = [str(x).zfill(2) for x in range(8,19)]
columns = ["RATE_" + year for year in years]
columns.append("MUNCOD")

disease_list = []

for file in all_files:
    file_name = file.split("\\")[-1]
    disease = file_name.split(".csv")[0]
    disease_df = pd.read_csv(file, sep=',', index_col=0)
    if(set(disease_df.columns) == set(columns)):
        disease_list.append(disease)
print(len(disease_list), " diseases")

313  diseases


In [7]:
final_df = pd.DataFrame()

for i in range(len(years)-1):
    col_year_suicide = "RATE_" + years[i+1]
    col_year_prev = "RATE_" + years[i]
    year_df = suicide[[col_year_prev, col_year_suicide, "MUNCOD"]]
    year_df = year_df.rename(columns={col_year_suicide: "RATE"})
    year_df = year_df.rename(columns={col_year_prev: "PREVIOUS"})
    for disease in disease_list:
        col_year_disease = "RATE_" + years[i]
        disease_df = pd.read_csv(path + disease + ".csv", sep=',', index_col=0)
        disease_df = disease_df[[col_year_disease, "MUNCOD"]]
        disease_df = disease_df.rename(columns={col_year_disease: disease})

        year_df = pd.merge(disease_df, year_df, left_on="MUNCOD", right_on="MUNCOD", how='right')
        
    final_df = pd.concat([final_df, year_df])

final_df.head()

,ÚLCERA_GÁSTRICA_E_DUODENAL,MUNCOD,VEIAS_VARICOSAS_DAS_EXTREMIDADES_INFERIORES,VARICELA_E_HERPES_ZOSTER,UROLITÍASE,TÉTANO_NEONATAL,TUBERC_INTEST_PERITÔNIO_GLÂNGL_MESENTÉRICOS,TUBERCULOSE_ÓSSEA_E_DAS_ARTICULAÇÕES,TUBERCULOSE_PULMONAR,TUBERCULOSE_MILIAR,...,ANCILOSTOMÍASE,AMEBÍASE,ALGUNS_TRANSTORNOS_ENVOLVENDO_MECANISMO_IMUNITÁRIO,AFECÇ_HEMORRÁG_E_OUTR_DOENÇ_SANG_E_ÓRG_HEMATOPOÉT,ACID_VASCULAR_CEREBR_NÃO_ESPEC_HEMORRÁG_OU_ISQUÊM,ACID_VASCULAR_CEREBR_ISQUÊM_TRANSIT_E_SÍNDR_CORREL,ABORTO_POR_RAZÕES_MÉDICAS,ABORTO_ESPONTÂNEO,PREVIOUS,RATE
0,28.481914,110001,40.688449,0.000000,142.409570,NaN,NaN,NaN,0.0,NaN,...,NaN,0.0,NaN,44.757293,40.688449,32.550759,NaN,126.134191,20.344224,8.212203
1,9.458389,110002,54.385737,0.000000,15.369882,NaN,0.0,NaN,0.0,0.0,...,NaN,0.0,0.0,1.182299,39.015855,0.000000,0.0,1.182299,9.458389,2.338060
2,0.000000,110003,14.755792,0.000000,29.511583,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000000,118.046333,0.000000,NaN,NaN,0.000000,14.936520
3,5.110972,110004,21.721631,0.000000,63.887150,NaN,NaN,NaN,0.0,0.0,...,NaN,NaN,0.0,3.833229,106.052669,3.833229,0.0,2.555486,5.110972,7.626311
4,5.958055,110005,41.706387,5.958055,113.203051,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,5.958055,95.328885,0.000000,0.0,5.958055,0.000000,0.000000


Get UF from MUNCOD

In [9]:
final_df['UF'] = final_df['MUNCOD'] / 10000
final_df['UF'] = final_df['UF'].astype(int)
final_df['UF'].replace(dict_uf_cod, inplace=True)
dummy = pd.get_dummies(final_df['UF'])
final_df = pd.concat([final_df, dummy], axis=1)
final_df = final_df.drop(['MUNCOD', 'UF'], axis=1)
final_df.head()

,ÚLCERA_GÁSTRICA_E_DUODENAL,VEIAS_VARICOSAS_DAS_EXTREMIDADES_INFERIORES,VARICELA_E_HERPES_ZOSTER,UROLITÍASE,TÉTANO_NEONATAL,TUBERC_INTEST_PERITÔNIO_GLÂNGL_MESENTÉRICOS,TUBERCULOSE_ÓSSEA_E_DAS_ARTICULAÇÕES,TUBERCULOSE_PULMONAR,TUBERCULOSE_MILIAR,TUBERCULOSE_DO_SISTEMA_NERVOSO,...,PR,RJ,RN,RO,RR,RS,SC,SE,SP,TO
0,28.481914,40.688449,0.000000,142.409570,NaN,NaN,NaN,0.0,NaN,0.0,...,0,0,0,1,0,0,0,0,0,0
1,9.458389,54.385737,0.000000,15.369882,NaN,0.0,NaN,0.0,0.0,NaN,...,0,0,0,1,0,0,0,0,0,0
2,0.000000,14.755792,0.000000,29.511583,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0
3,5.110972,21.721631,0.000000,63.887150,NaN,NaN,NaN,0.0,0.0,NaN,...,0,0,0,1,0,0,0,0,0,0
4,5.958055,41.706387,5.958055,113.203051,NaN,NaN,NaN,0.0,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0


Define X and y

In [15]:
percentage_valid_values = 0.8
num_feat = 50
corr_min_value = 0.05

print("Shape before null values drop:", final_df.shape)
N = int(final_df.shape[1]*percentage_valid_values)
print("Minimum non-null values: " + str(N) + "/" + str(final_df.shape[1]))
final_zeros_df = final_df.dropna(thresh=N) # At least N non null items

X = final_zeros_df.drop(columns="RATE")
X = X.fillna(0)
y = final_zeros_df["RATE"]

X.shape

Shape before null values drop: (53810, 342)
Minimum non-null values: 273/342


(11920, 341)

Define some functions

In [68]:
def calcUpMetrics(y_pred,y_test,X_test):
    up_df = pd.DataFrame({"Pred": y_pred, "Real": y_test, "Previous": X_test["PREVIOUS"]})
    up_df["UP"] = up_df["Previous"] < up_df["Real"]
    up_df["UP_PRED"] = up_df["Previous"] < up_df["Pred"]
    up_df["UP"] = up_df["UP"].astype(int)
    up_df["UP_PRED"] = up_df["UP_PRED"].astype(int)
    return metrics.accuracy_score(up_df["UP"], up_df["UP_PRED"])

In [123]:
def get_average_feat_importance(feature_importance_list):
    final_feat_df = pd.DataFrame()
    for i, feat_df in enumerate(feature_importance_list):
        feat_df = feat_df.rename(columns={"Importance": i})
        if final_feat_df.empty:
            final_feat_df = feat_df
        else:
            final_feat_df = pd.merge(final_feat_df, feat_df, on="Feature")
    final_feat_df["Avg_importance"] = final_feat_df.sum(axis=1)/(final_feat_df.shape[1] -1)
    final_feat_df = final_feat_df[["Feature", "Avg_importance"]]
    return final_feat_df

Create K-folds

In [48]:
def cor_feature_selector(X,y,corr_min_value):
    cor_list = []
    for i in list(X.columns):
        cor = np.corrcoef(X[i], y)[0,1]
        cor_list.append([i, cor])
    cor_feature = [x[0] for x in cor_list if abs(x[1]) > corr_min_value]
    print(len(cor_feature), "selected features (correlation)")
    return cor_feature

First model: no bayesian optimization

In [88]:
from sklearn.model_selection import KFold
start = time.time()
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)
scores = pd.DataFrame({"rmse_train": [], "rmse_test": [], "rmse_baseline_train": [], "rmse_baseline_test": [], "up_down_train": [], "up_down_test": []})
feature_importance_list = []
count = 1
for train_index, test_index in kf.split(X):
    print("=============== RUN {} ===============".format(count))
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    print("# Feature Selection: correlation")
    selected_cor_features = cor_feature_selector(X_train,y_train,corr_min_value)
    X_train = X_train[selected_cor_features]
    X_test = X_test[selected_cor_features]
    
    print("# Fitting the model")
    regressor = RandomForestRegressor(random_state=42)
    regressor.fit(X_train,y_train)
    
    print("# Making predictions")
    y_pred = regressor.predict(X_test) 
    y_pred_train = regressor.predict(X_train)
    
    print("# Calculating metrics")
    rmse_train = np.sqrt(metrics.mean_squared_error(y_pred_train,y_train))
    rmse_test = np.sqrt(metrics.mean_squared_error(y_pred,y_test))
    rmse_baseline_train = np.sqrt(metrics.mean_squared_error(X_train["PREVIOUS"],y_train))
    rmse_baseline_test = np.sqrt(metrics.mean_squared_error(X_test["PREVIOUS"],y_test))
    up_down_train = calcUpMetrics(y_pred_train,y_train,X_train)
    up_down_test = calcUpMetrics(y_pred,y_test,X_test)
                                 
    print("RMSE Train:", rmse_train)
    print("RMSE Test:", rmse_test)
    print("RMSE Baseline Train:", rmse_baseline_train)
    print("RMSE Baseline Test:", rmse_baseline_test)
    print("Up/Down Accuracy Train:", up_down_train)
    print("Up/Down Accuracy Test:", up_down_test)
    scores = scores.append({'rmse_train':rmse_train, 'rmse_test':rmse_test, "rmse_baseline_train": rmse_baseline_train, "rmse_baseline_test": rmse_baseline_test, "up_down_train": up_down_train, "up_down_test": up_down_test},ignore_index=True)
    
    print("# Get Feature Importance")
    importance = regressor.feature_importances_
    feature_importance_list.append(pd.DataFrame({"Feature": X_train.columns, "Importance": importance}).sort_values(by="Importance", ascending=False))
    
    print("\n")
    count += 1
print("=============== SUMMARY ===============")
print("# Average Metrics")
print(scores.mean())
print("# Average Feature Importance")
final_feat_df = get_average_feat_importance(feature_importance_list)
print(final_feat_df.head(20))
end = time.time()

print("\nTotal run time:", end-start)

=============== RUN 1 ===============
# Feature Selection: correlation
121 selected features (correlation)
# Fitting the model
# Making predictions
# Calculating metrics
RMSE Train: 1.6427325700322828
RMSE Test: 4.636173203018522
RMSE Baseline Train: 5.587001133045161
RMSE Baseline Test: 5.708701552449413
Up/Down Accuracy Train: 0.8623112416107382
Up/Down Accuracy Test: 0.6455536912751678
# Get Feature Importance


=============== RUN 2 ===============
# Feature Selection: correlation
111 selected features (correlation)
# Fitting the model
# Making predictions
# Calculating metrics
RMSE Train: 1.689121384056114
RMSE Test: 4.276097296046732
RMSE Baseline Train: 5.6053569012030104
RMSE Baseline Test: 5.636266148619417
Up/Down Accuracy Train: 0.8622063758389261
Up/Down Accuracy Test: 0.6661073825503355
# Get Feature Importance


=============== RUN 3 ===============
# Feature Selection: correlation
119 selected features (correlation)
# Fitting the model
# Making predictions
# Calculating 

Second model: with bayesian optimization

In [99]:
from sklearn.model_selection import KFold
start = time.time()
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)
scores = pd.DataFrame({"rmse_train": [], "rmse_test": [], "rmse_baseline_train": [], "rmse_baseline_test": [], "up_down_train": [], "up_down_test": []})
feature_importance_list = []
count = 1
for train_index, test_index in kf.split(X):
    if(count > 1):
        break # delete this later
    print("=============== RUN {} ===============".format(count))
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    print("# Feature Selection: correlation")
    selected_cor_features = cor_feature_selector(X_train,y_train,corr_min_value)
    X_train = X_train[selected_cor_features]
    X_test = X_test[selected_cor_features]
    
    print("# Bayesian Optimization")
    def train_model(params):
        n_estimators = params[0]
        max_depth = params[1]
        min_samples_leaf = params[2]
        max_features = params[3]

        xf_train_params,xf_val,yf_train_params,yf_val = train_test_split(X_train,y_train,test_size=0.33,random_state=42)

        model = RandomForestRegressor(n_jobs=-1, random_state=42,n_estimators=n_estimators, 
                                      max_depth=max_depth,min_samples_leaf=min_samples_leaf,max_features=max_features)

        model.fit(xf_train_params,yf_train_params)
        yf_pred = model.predict(xf_val)

        return np.sqrt(metrics.mean_squared_error(yf_val, yf_pred))
    space = [
        (5,1000), #n_estimators
        (3,30), #max_depth
        (2,200), #min_samples_leaf
        (0.25,1.00) #max_features
    ]
    resultado_gp = gp_minimize(train_model, space, random_state=42, verbose=0, n_calls=30, n_random_starts=10)
    
    print("# Fitting the model")
    regressor = RandomForestRegressor(n_jobs=-1, random_state=42,n_estimators=resultado_gp.x[0], 
                                  max_depth=resultado_gp.x[1],min_samples_leaf=resultado_gp.x[2],max_features=resultado_gp.x[3])
    regressor.fit(X_train,y_train)
    
    print("# Making predictions")
    y_pred = regressor.predict(X_test) 
    y_pred_train = regressor.predict(X_train)
    
    print("# Calculating metrics")
    rmse_train = np.sqrt(metrics.mean_squared_error(y_pred_train,y_train))
    rmse_test = np.sqrt(metrics.mean_squared_error(y_pred,y_test))
    rmse_baseline_train = np.sqrt(metrics.mean_squared_error(X_train["PREVIOUS"],y_train))
    rmse_baseline_test = np.sqrt(metrics.mean_squared_error(X_test["PREVIOUS"],y_test))
    up_down_train = calcUpMetrics(y_pred_train,y_train,X_train)
    up_down_test = calcUpMetrics(y_pred,y_test,X_test)
                                 
    print("RMSE Train:", rmse_train)
    print("RMSE Test:", rmse_test)
    print("RMSE Baseline Train:", rmse_baseline_train)
    print("RMSE Baseline Test:", rmse_baseline_test)
    print("Up/Down Accuracy Train:", up_down_train)
    print("Up/Down Accuracy Test:", up_down_test)
    scores = scores.append({'rmse_train':rmse_train, 'rmse_test':rmse_test, "rmse_baseline_train": rmse_baseline_train, "rmse_baseline_test": rmse_baseline_test, "up_down_train": up_down_train, "up_down_test": up_down_test},ignore_index=True)
    
    print("# Get Feature Importance")
    importance = regressor.feature_importances_
    feature_importance_list.append(pd.DataFrame({"Feature": X_train.columns, "Importance": importance}).sort_values(by="Importance", ascending=False))
    
    print("\n")
    count += 1
print("=============== SUMMARY ===============")
print("# Average Metrics")
print(scores.mean())
print("# Average Feature Importance")
final_feat_df = get_average_feat_importance(feature_importance_list)
print(final_feat_df.head(20))
end = time.time()

print("\nTotal run time:", end-start)

=============== RUN 1 ===============
# Feature Selection: correlation
121 selected features (correlation)
# Bayesian Optimization
# Fitting the model
# Making predictions
# Calculating metrics
RMSE Train: 1.8706874725355613
RMSE Test: 4.5559191861286665
RMSE Baseline Train: 5.587001133045161
RMSE Baseline Test: 5.708701552449413
Up/Down Accuracy Train: 0.8560192953020134
Up/Down Accuracy Test: 0.6535234899328859
# Get Feature Importance


=============== SUMMARY ===============
rmse_train             1.870687
rmse_test              4.555919
rmse_baseline_train    5.587001
rmse_baseline_test     5.708702
up_down_train          0.856019
up_down_test           0.653523
dtype: float64


Third model: with RFE feature selection

In [105]:
def rfe_feature_selector(X,y,n_features_to_select):
    rfe_selector = RFE(estimator=RandomForestRegressor(random_state=42), step=10, verbose=0,n_features_to_select=n_features_to_select)
    rfe_selector.fit(X, y)
    rfe_support = rfe_selector.get_support()
    rfe_feature = X.loc[:,rfe_support].columns.tolist()
    print(len(rfe_feature), 'selected features (RFE)')
    return rfe_feature

In [106]:
from sklearn.model_selection import KFold
start = time.time()
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)
scores = pd.DataFrame({"rmse_train": [], "rmse_test": [], "rmse_baseline_train": [], "rmse_baseline_test": [], "up_down_train": [], "up_down_test": []})
feature_importance_list = []
count = 1
for train_index, test_index in kf.split(X):
    if(count > 1):
        break # delete this later
    print("=============== RUN {} ===============".format(count))
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    print("# Feature Selection: correlation")
    selected_cor_features = cor_feature_selector(X_train,y_train,corr_min_value)
    X_train = X_train[selected_cor_features]
    X_test = X_test[selected_cor_features]
    
    print("# Feature Selection: RFE")
    selected_rfe_features = rfe_feature_selector(X_train,y_train,num_feat)
    X_train = X_train[selected_rfe_features]
    X_test = X_test[selected_rfe_features]
    
    print("# Bayesian Optimization")
    def train_model(params):
        n_estimators = params[0]
        max_depth = params[1]
        min_samples_leaf = params[2]
        max_features = params[3]

        xf_train_params,xf_val,yf_train_params,yf_val = train_test_split(X_train,y_train,test_size=0.33,random_state=42)

        model = RandomForestRegressor(n_jobs=-1, random_state=42,n_estimators=n_estimators, 
                                      max_depth=max_depth,min_samples_leaf=min_samples_leaf,max_features=max_features)

        model.fit(xf_train_params,yf_train_params)
        yf_pred = model.predict(xf_val)

        return np.sqrt(metrics.mean_squared_error(yf_val, yf_pred))
    space = [
        (5,1000), #n_estimators
        (3,30), #max_depth
        (2,200), #min_samples_leaf
        (0.25,1.00) #max_features
    ]
    resultado_gp = gp_minimize(train_model, space, random_state=42, verbose=0, n_calls=30, n_random_starts=10)
    
    print("# Fitting the model")
    regressor = RandomForestRegressor(n_jobs=-1, random_state=42,n_estimators=resultado_gp.x[0], 
                                  max_depth=resultado_gp.x[1],min_samples_leaf=resultado_gp.x[2],max_features=resultado_gp.x[3])
    regressor.fit(X_train,y_train)
    
    print("# Making predictions")
    y_pred = regressor.predict(X_test) 
    y_pred_train = regressor.predict(X_train)
    
    print("# Calculating metrics")
    rmse_train = np.sqrt(metrics.mean_squared_error(y_pred_train,y_train))
    rmse_test = np.sqrt(metrics.mean_squared_error(y_pred,y_test))
    rmse_baseline_train = np.sqrt(metrics.mean_squared_error(X_train["PREVIOUS"],y_train))
    rmse_baseline_test = np.sqrt(metrics.mean_squared_error(X_test["PREVIOUS"],y_test))
    up_down_train = calcUpMetrics(y_pred_train,y_train,X_train)
    up_down_test = calcUpMetrics(y_pred,y_test,X_test)
                                 
    print("RMSE Train:", rmse_train)
    print("RMSE Test:", rmse_test)
    print("RMSE Baseline Train:", rmse_baseline_train)
    print("RMSE Baseline Test:", rmse_baseline_test)
    print("Up/Down Accuracy Train:", up_down_train)
    print("Up/Down Accuracy Test:", up_down_test)
    scores = scores.append({'rmse_train':rmse_train, 'rmse_test':rmse_test, "rmse_baseline_train": rmse_baseline_train, "rmse_baseline_test": rmse_baseline_test, "up_down_train": up_down_train, "up_down_test": up_down_test},ignore_index=True)
    
    print("# Get Feature Importance")
    importance = regressor.feature_importances_
    feature_importance_list.append(pd.DataFrame({"Feature": X_train.columns, "Importance": importance}).sort_values(by="Importance", ascending=False))
    
    print("\n")
    count += 1
print("=============== SUMMARY ===============")
print("# Average Metrics")
print(scores.mean())
print("# Average Feature Importance")
final_feat_df = get_average_feat_importance(feature_importance_list)
print(final_feat_df.head(20))
end = time.time()

print("\nTotal run time:", end-start)

=============== RUN 1 ===============
# Feature Selection: correlation
121 selected features (correlation)
# Feature Selection: RFE
50 selected features (RFE)
# Bayesian Optimization
# Fitting the model
# Making predictions
# Calculating metrics
RMSE Train: 1.930398142725495
RMSE Test: 4.570381192405489
RMSE Baseline Train: 5.587001133045161
RMSE Baseline Test: 5.708701552449413
Up/Down Accuracy Train: 0.855494966442953
Up/Down Accuracy Test: 0.6535234899328859
# Get Feature Importance


=============== SUMMARY ===============
rmse_train             1.930398
rmse_test              4.570381
rmse_baseline_train    5.587001
rmse_baseline_test     5.708702
up_down_train          0.855495
up_down_test           0.653523
dtype: float64
Total run time: 1549.431699514389
